In [49]:
# Bibliotecas utilizadas
import os
import pandas as pd
import pyodbc
from datetime import datetime, timedelta
import win32com.client as win32
import warnings

# Pega usuário de rede
usuario = os.getenv('USERNAME')

# Tira mensagens de warning
warnings.filterwarnings('ignore')
 
# 1. Caminho para o arquivo com credenciais
path = fr'C:\Users\{usuario}\OneDrive - CAIXA Consórcio\Documentos\SENHA_BANCO_DADOS'
os.chdir(path)

# 2. Carrega credenciais
df_senhas = pd.read_excel('SENHAS.xlsx')
server, database, username, password = df_senhas.iloc[0, 0:4]
 
# 3. Conexão com o SQL Server
conn = pyodbc.connect(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
)
 
# 4. Calcular último dia do mês anterior
hoje = datetime.today()
primeiro_dia_mes_atual = datetime(hoje.year, hoje.month, 1)
ultimo_dia_mes_anterior = primeiro_dia_mes_atual - timedelta(days=1)
data_referencia = ultimo_dia_mes_anterior.strftime('%Y-%m-%d')
 
# 5. Queries
query_valor = """
SELECT 
    Grupo,
    Produto,
    Tipo_Venda,
    Faixa_Credito,
    ISNULL(Adimplente, 0) AS Adimplente,
    ISNULL(Inadimplente, 0) AS Inadimplente,
    ROUND(100 * (CAST(ISNULL(Inadimplente, 0) AS FLOAT) / NULLIF(ISNULL(Adimplente, 0) + ISNULL(Inadimplente, 0), 0)), 1) AS Perc_Inadimplente
FROM (
    SELECT
        a.Grupo,
        b.Produto,
        b.Tipo_Venda,
        b.Faixa_Credito,
        a.ST_Adimplencia,
        a.VL_Bem
    FROM [dbo].[FT0015_CarteiraCotas] a
    LEFT JOIN [dbo].[FT0004_GruposAtual] b ON a.Grupo = b.Grupo
    WHERE a.DT_Referencia = '{0}'
) src
PIVOT (
    SUM(VL_Bem)
    FOR ST_Adimplencia IN ([Adimplente], [Inadimplente])
) AS pvt
ORDER BY Perc_Inadimplente DESC
"""
 
query_cotas = """
SELECT 
    pvt.Grupo,
    pvt.Produto,
    pvt.Tipo_Venda,
    pvt.Faixa_Credito,
    ISNULL(pvt.Adimplente, 0) AS Adimplente,
    ISNULL(pvt.Inadimplente, 0) AS Inadimplente,
    ROUND(
        100 * (CAST(ISNULL(pvt.Inadimplente, 0) AS FLOAT) / 
        NULLIF(ISNULL(pvt.Adimplente, 0) + ISNULL(pvt.Inadimplente, 0), 0)),
        1
    ) AS Perc_Inadimplente,
    qtd.Qtd_Pessoas
FROM (
    SELECT
        a.Grupo,
        b.Produto,
        b.Tipo_Venda,
        b.Faixa_Credito,
        UPPER(LTRIM(RTRIM(a.ST_Adimplencia))) AS ST_Adimplencia,
        a.ID_Cota
    FROM [dbo].[FT0015_CarteiraCotas] a
    LEFT JOIN [dbo].[FT0004_GruposAtual] b ON a.Grupo = b.Grupo
    WHERE a.DT_Referencia = '{0}'
) AS src
PIVOT (
    COUNT(ID_Cota)
    FOR ST_Adimplencia IN ([ADIMPLENTE], [INADIMPLENTE])
) AS pvt
LEFT JOIN (
    SELECT
        a.Grupo,
        b.Produto,
        b.Tipo_Venda,
        b.Faixa_Credito,
        COUNT(DISTINCT c.CD_InscricaoNacional) AS Qtd_Pessoas
    FROM [dbo].[FT0015_CarteiraCotas] a
    LEFT JOIN [dbo].[FT0004_GruposAtual] b ON a.Grupo = b.Grupo
    LEFT JOIN [dbo].[DM0013_Pessoas] c ON a.ID_Pessoa = c.ID_Pessoa
    WHERE a.DT_Referencia = '{0}'
    GROUP BY a.Grupo, b.Produto, b.Tipo_Venda, b.Faixa_Credito
) AS qtd
ON pvt.Grupo = qtd.Grupo
AND pvt.Produto = qtd.Produto
AND pvt.Tipo_Venda = qtd.Tipo_Venda
AND pvt.Faixa_Credito = qtd.Faixa_Credito
ORDER BY Perc_Inadimplente DESC
"""
 
# 6. Executar queries
df_valor = pd.read_sql(query_valor.format(data_referencia), conn)
df_cotas = pd.read_sql(query_cotas.format(data_referencia), conn)
 
# 7. Formatar colunas de valor em milhões
df_valor['Adimplente'] = df_valor['Adimplente'].astype(float) / 1_000_000
df_valor['Adimplente'] = df_valor['Adimplente'].apply(lambda x: f"{x:,.1f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
 
df_valor['Inadimplente'] = df_valor['Inadimplente'].astype(float) / 1_000_000
df_valor['Inadimplente'] = df_valor['Inadimplente'].apply(lambda x: f"{x:,.1f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
 
df_valor['Perc_Inadimplente'] = df_valor['Perc_Inadimplente'].apply(lambda x: f"{x:.1f}%".replace('.', ','))
 
# 8. Renomear colunas
col_novo = f"% Inadimplente ({data_referencia})"
df_valor.rename(columns={
    'Tipo_Venda': 'Tipo Grupo',
    'Faixa_Credito': 'Faixa de Crédito',
    'Adimplente': 'R$ MM Adimplente',
    'Inadimplente': 'R$ MM Inadimplente',
    'Perc_Inadimplente': col_novo
}, inplace=True)
 
df_cotas.rename(columns={
    'Tipo_Venda': 'Tipo Grupo',
    'Faixa_Credito': 'Faixa de Crédito',
    'Adimplente': 'Qtd. Cotas Adimplentes',
    'Inadimplente': 'Qtd. Cotas Inadimplentes',
    'Qtd_Pessoas': 'Qtd. Clientes',
    'Perc_Inadimplente': col_novo
}, inplace=True)
 
# 9. Consultar e adicionar mês anterior
data_referencia_anterior = (datetime.strptime(data_referencia, '%Y-%m-%d') - timedelta(days=1)).replace(day=1) - timedelta(days=1)
data_referencia_anterior = data_referencia_anterior.strftime('%Y-%m-%d')
 
df_valor_anterior = pd.read_sql(query_valor.format(data_referencia_anterior), conn)
df_valor_anterior.rename(columns={
    'Tipo_Venda': 'Tipo Grupo',
    'Faixa_Credito': 'Faixa de Crédito',
    'Perc_Inadimplente': f'% Inadimplente ({data_referencia_anterior})'
}, inplace=True)
df_valor = pd.merge(df_valor, df_valor_anterior[['Grupo', 'Produto', 'Tipo Grupo', 'Faixa de Crédito', f'% Inadimplente ({data_referencia_anterior})']],
                    on=['Grupo', 'Produto', 'Tipo Grupo', 'Faixa de Crédito'], how='left')
 
df_cotas_anterior = pd.read_sql(query_cotas.format(data_referencia_anterior), conn)
df_cotas_anterior.rename(columns={
    'Tipo_Venda': 'Tipo Grupo',
    'Faixa_Credito': 'Faixa de Crédito',
    'Perc_Inadimplente': f'% Inadimplente ({data_referencia_anterior})'
}, inplace=True)
df_cotas = pd.merge(df_cotas, df_cotas_anterior[['Grupo', 'Produto', 'Tipo Grupo', 'Faixa de Crédito', f'% Inadimplente ({data_referencia_anterior})']],
                    on=['Grupo', 'Produto', 'Tipo Grupo', 'Faixa de Crédito'], how='left')


# 10. Análise de Score médio de crédito na venda e contemplação por grupo
def trata_documento(val):
    val = float(val)
    val = str(val).split('.')[0]
    return(val)
# Carregando tabela
script = """
select
    a.ID_Cota,
    a.Grupo,
    c.CD_InscricaoNacional as CPF_CNPJ,
    a.DT_Alocacao,
    b.DT_Contemplacao,
    a.Desclassificado
from [dbo].[FT0002_VendaAlocacoes] a
    left join [dbo].[FT0019_EntregaBem] b on a.ID_Cota = b.ID_Cota
    left join [dbo].[DM0013_Pessoas] c on a.ID_Pessoa = c.ID_Pessoa
where a.ST_Contrato = 'Ativo'
"""
df_dw = pd.read_sql(script, conn)

# Tratando colunas de data
df_dw['DT_Alocacao'] = pd.to_datetime(df_dw['DT_Alocacao'], format='%Y-%m-%d')
df_dw['DT_Contemplacao'] = pd.to_datetime(df_dw['DT_Contemplacao'], format='%Y-%m-%d')
df_dw['CPF_CNPJ'] = df_dw['CPF_CNPJ'].apply(trata_documento)

# 10.1. Base de Scores
path = fr'C:\Users\{usuario}\CAIXA Consórcio\Risco de Crédito e Antifraude - Documentos\MODELOS\MODELAGEM\SCORE_CREDITO_BOA_VISTA\BASES_HISTORICAS'
os.chdir(path)
df_score = pd.read_csv('score_credito_full.csv', delimiter=";", encoding='latin1')
df_score['Data'] = pd.to_datetime(df_score['Data'], format='%Y-%m-%d')
df_score['CPF_CNPJ'] = df_score['CPF_CNPJ'].apply(trata_documento)
df_score = df_score[['Data', 'CPF_CNPJ', 'Score_Credito']]
df_score = df_score[df_score['Score_Credito'] > 2]

# 10.2. Coletando Scores na Venda
df_venda = df_dw[['ID_Cota', 'Grupo', 'CPF_CNPJ', 'DT_Alocacao']]
df_venda = df_venda.merge(df_score, how='left', on=['CPF_CNPJ'])
df_venda = df_venda[df_venda['Score_Credito'].notna()]
df_venda['Variacao_Datas'] = abs(df_venda['Data'] - df_venda['DT_Alocacao']).dt.days
df_venda = df_venda.sort_values(by=['Variacao_Datas'], ascending=True)
df_venda = df_venda.drop_duplicates(subset=['ID_Cota'])
df_venda = df_venda[['Grupo', 'Score_Credito']].groupby(by=['Grupo']).mean().reset_index(level=['Grupo'])
df_venda = df_venda.rename(columns={'Score_Credito': 'Score de Crédito (Venda)'})
df_venda['Score de Crédito (Venda)'] = df_venda['Score de Crédito (Venda)'].round(0)

# 10.3. Coletando Scores na Venda
df_contemplacao = df_dw[df_dw['Desclassificado'] == 'Não']
df_contemplacao = df_contemplacao[['ID_Cota', 'Grupo', 'CPF_CNPJ', 'DT_Contemplacao']]
df_contemplacao = df_contemplacao.merge(df_score, how='left', on=['CPF_CNPJ'])
df_contemplacao = df_contemplacao[df_contemplacao['Score_Credito'].notna()]
df_contemplacao['Variacao_Datas'] = abs(df_contemplacao['Data'] - df_contemplacao['DT_Contemplacao']).dt.days
df_contemplacao = df_contemplacao.sort_values(by=['Variacao_Datas'], ascending=True)
df_contemplacao = df_contemplacao.drop_duplicates(subset=['ID_Cota'])
df_contemplacao = df_contemplacao[['Grupo', 'Score_Credito']].groupby(by=['Grupo']).mean().reset_index(level=['Grupo'])
df_contemplacao = df_contemplacao.rename(columns={'Score_Credito': 'Score de Crédito (Contemplação)'})
df_contemplacao['Score de Crédito (Contemplação)'] = df_contemplacao['Score de Crédito (Contemplação)'].round(0)

# 10.4. Juntando as duas tabelas
df_venda = df_venda.merge(df_contemplacao, how='left', on=['Grupo'])

# 11. Juntando as tabelas 
df_cotas = df_cotas.merge(df_venda, how='left', on=['Grupo'])
df_valor = df_valor.merge(df_venda, how='left', on=['Grupo'])


In [50]:
# Pega usuário de rede
usuario = os.getenv('USERNAME')
 
# Tira mensagens de warning
warnings.filterwarnings('ignore')
 
# 1. Caminho para o arquivo com credenciais
path = fr'C:\Users\{usuario}\OneDrive - CAIXA Consórcio\Documentos\SENHA_BANCO_DADOS'
os.chdir(path)
 
# 2. Carrega credenciais
df_senhas = pd.read_excel('SENHAS.xlsx')
server, database, username, password = df_senhas.iloc[0, 0:4]
 
# 3. Conexão com o SQL Server
conn = pyodbc.connect(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
)
 
# 4. Calcular último dia do mês anterior
hoje = datetime.today()
primeiro_dia_mes_atual = datetime(hoje.year, hoje.month, 1)
ultimo_dia_mes_anterior = primeiro_dia_mes_atual - timedelta(days=1)
data_referencia = ultimo_dia_mes_anterior.strftime('%Y-%m-%d')

In [51]:
def trata_documento(val):
    val = float(val)
    val = str(val).split('.')[0]
    return(val)

def percentual_para_float(perc_str):
    if pd.isna(perc_str):
        return 0.0
    if isinstance(perc_str, float):
        return perc_str
    return float(perc_str.replace('%', '').replace(',', '.'))

In [52]:
# 5. Queries
query_valor = """
SELECT 
    Grupo,
    Produto,
    Tipo_Venda,
    Faixa_Credito,
    ISNULL(Adimplente, 0) AS Adimplente,
    ISNULL(Inadimplente, 0) AS Inadimplente,
    ROUND(100 * (CAST(ISNULL(Inadimplente, 0) AS FLOAT) / NULLIF(ISNULL(Adimplente, 0) + ISNULL(Inadimplente, 0), 0)), 1) AS Perc_Inadimplente
FROM (
    SELECT
        a.Grupo,
        b.Produto,
        b.Tipo_Venda,
        b.Faixa_Credito,
        a.ST_Adimplencia,
        a.VL_Bem
    FROM [dbo].[FT0015_CarteiraCotas] a
    LEFT JOIN [dbo].[FT0004_GruposAtual] b ON a.Grupo = b.Grupo
    WHERE a.DT_Referencia = '{0}'
) src
PIVOT (
    SUM(VL_Bem)
    FOR ST_Adimplencia IN ([Adimplente], [Inadimplente])
) AS pvt
ORDER BY Perc_Inadimplente DESC
"""
 
query_cotas = """
SELECT 
    pvt.Grupo,
    pvt.Produto,
    pvt.Tipo_Venda,
    pvt.Faixa_Credito,
    ISNULL(pvt.Adimplente, 0) AS Adimplente,
    ISNULL(pvt.Inadimplente, 0) AS Inadimplente,
    ROUND(
        100 * (CAST(ISNULL(pvt.Inadimplente, 0) AS FLOAT) /
        NULLIF(ISNULL(pvt.Adimplente, 0) + ISNULL(pvt.Inadimplente, 0), 0)),
        1
    ) AS Perc_Inadimplente,
    qtd.Qtd_Pessoas
FROM (
    SELECT
        a.Grupo,
        b.Produto,
        b.Tipo_Venda,
        b.Faixa_Credito,
        UPPER(LTRIM(RTRIM(a.ST_Adimplencia))) AS ST_Adimplencia,
        a.ID_Cota
    FROM [dbo].[FT0015_CarteiraCotas] a
    LEFT JOIN [dbo].[FT0004_GruposAtual] b ON a.Grupo = b.Grupo
    WHERE a.DT_Referencia = '{0}'
) AS src
PIVOT (
    COUNT(ID_Cota)
    FOR ST_Adimplencia IN ([ADIMPLENTE], [INADIMPLENTE])
) AS pvt
LEFT JOIN (
    SELECT
        a.Grupo,
        b.Produto,
        b.Tipo_Venda,
        b.Faixa_Credito,
        COUNT(DISTINCT c.CD_InscricaoNacional) AS Qtd_Pessoas
    FROM [dbo].[FT0015_CarteiraCotas] a
    LEFT JOIN [dbo].[FT0004_GruposAtual] b ON a.Grupo = b.Grupo
    LEFT JOIN [dbo].[DM0013_Pessoas] c ON a.ID_Pessoa = c.ID_Pessoa
    WHERE a.DT_Referencia = '{0}'
    GROUP BY a.Grupo, b.Produto, b.Tipo_Venda, b.Faixa_Credito
) AS qtd
ON pvt.Grupo = qtd.Grupo
AND pvt.Produto = qtd.Produto
AND pvt.Tipo_Venda = qtd.Tipo_Venda
AND pvt.Faixa_Credito = qtd.Faixa_Credito
ORDER BY Perc_Inadimplente DESC
"""
 
# 6. Executar queries
df_valor = pd.read_sql(query_valor.format(data_referencia), conn)
df_cotas = pd.read_sql(query_cotas.format(data_referencia), conn)

In [53]:
# 7. Formatar colunas de valor em milhões
df_valor['Adimplente'] = df_valor['Adimplente'].astype(float) / 1_000_000
df_valor['Adimplente'] = df_valor['Adimplente'].apply(lambda x: f"{x:,.1f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
 
df_valor['Inadimplente'] = df_valor['Inadimplente'].astype(float) / 1_000_000
df_valor['Inadimplente'] = df_valor['Inadimplente'].apply(lambda x: f"{x:,.1f}".replace(',', 'X').replace('.', ',').replace('X', '.'))

df_valor['Perc_Inadimplente'] = df_valor['Perc_Inadimplente'].apply(lambda x: f"{x:.1f}".replace('.', ','))

In [54]:
# 8. Renomear colunas
col_novo = f"% Inadimplente ({data_referencia})"
df_valor.rename(columns={
    'Tipo_Venda': 'Tipo Grupo',
    'Faixa_Credito': 'Faixa de Crédito',
    'Adimplente': 'R$ MM Adimplente',
    'Inadimplente': 'R$ MM Inadimplente',
    'Perc_Inadimplente': col_novo
}, inplace=True)
 
df_cotas.rename(columns={
    'Tipo_Venda': 'Tipo Grupo',
    'Faixa_Credito': 'Faixa de Crédito',
    'Adimplente': 'Qtd. Cotas Adimplentes',
    'Inadimplente': 'Qtd. Cotas Inadimplentes',
    'Qtd_Pessoas': 'Qtd. Clientes',
    'Perc_Inadimplente': col_novo
}, inplace=True)

In [55]:
# 9. Consultar e adicionar mês anterior
data_referencia_anterior = (datetime.strptime(data_referencia, '%Y-%m-%d') - timedelta(days=1)).replace(day=1) - timedelta(days=1)
data_referencia_anterior = data_referencia_anterior.strftime('%Y-%m-%d')

In [56]:
grupos_mes_atual_cotas = nomes = df_cotas['Grupo'].dropna().unique().tolist()
lista_grupos_cotas = ', '.join(f"'{grupo}'" for grupo in grupos_mes_atual_cotas)

query_cotas_anterior = f"""
SELECT
    pvt.Grupo,
    pvt.Produto,
    pvt.Tipo_Venda,
    pvt.Faixa_Credito,
    ISNULL(pvt.Adimplente, 0) AS Adimplente,
    ISNULL(pvt.Inadimplente, 0) AS Inadimplente,
    ROUND(
        100 * (CAST(ISNULL(pvt.Inadimplente, 0) AS FLOAT) /
        NULLIF(ISNULL(pvt.Adimplente, 0) + ISNULL(pvt.Inadimplente, 0), 0)),
        1
    ) AS Perc_Inadimplente,
    qtd.Qtd_Pessoas
FROM (
    SELECT
        a.Grupo,
        b.Produto,
        b.Tipo_Venda,
        b.Faixa_Credito,
        UPPER(LTRIM(RTRIM(a.ST_Adimplencia))) AS ST_Adimplencia,
        a.ID_Cota
    FROM [dbo].[FT0015_CarteiraCotas] a
    LEFT JOIN [dbo].[FT0004_GruposAtual] b ON a.Grupo = b.Grupo
    WHERE a.DT_Referencia = '{data_referencia_anterior}'
) AS src
PIVOT (
    COUNT(ID_Cota)
    FOR ST_Adimplencia IN ([ADIMPLENTE], [INADIMPLENTE])
) AS pvt
LEFT JOIN (
    SELECT
        a.Grupo,
        b.Produto,
        b.Tipo_Venda,
        b.Faixa_Credito,
        COUNT(DISTINCT c.CD_InscricaoNacional) AS Qtd_Pessoas
    FROM [dbo].[FT0015_CarteiraCotas] a
    LEFT JOIN [dbo].[FT0004_GruposAtual] b ON a.Grupo = b.Grupo
    LEFT JOIN [dbo].[DM0013_Pessoas] c ON a.ID_Pessoa = c.ID_Pessoa
    WHERE a.DT_Referencia = '{data_referencia_anterior}'
    GROUP BY a.Grupo, b.Produto, b.Tipo_Venda, b.Faixa_Credito
) AS qtd
ON pvt.Grupo = qtd.Grupo
AND pvt.Produto = qtd.Produto
AND pvt.Tipo_Venda = qtd.Tipo_Venda
AND pvt.Faixa_Credito = qtd.Faixa_Credito
WHERE pvt.Grupo IN ({lista_grupos_cotas}
)
ORDER BY Perc_Inadimplente DESC
"""

In [57]:
grupos_mes_atual_valor = nomes = df_valor['Grupo'].dropna().unique().tolist()
lista_grupos_valor = ', '.join(f"'{grupo}'" for grupo in grupos_mes_atual_valor)

query_valor_anterior = f"""
SELECT
    Grupo,
    Produto,
    Tipo_Venda,
    Faixa_Credito,
    ISNULL(Adimplente, 0) AS Adimplente,
    ISNULL(Inadimplente, 0) AS Inadimplente,
    ROUND(100 * (CAST(ISNULL(Inadimplente, 0) AS FLOAT) / NULLIF(ISNULL(Adimplente, 0) + ISNULL(Inadimplente, 0), 0)), 1) AS Perc_Inadimplente
FROM (
    SELECT
        a.Grupo,
        b.Produto,
        b.Tipo_Venda,
        b.Faixa_Credito,
        a.ST_Adimplencia,
        a.VL_Bem
    FROM [dbo].[FT0015_CarteiraCotas] a
    LEFT JOIN [dbo].[FT0004_GruposAtual] b ON a.Grupo = b.Grupo
    WHERE a.DT_Referencia = '{data_referencia_anterior}'
    AND a.Grupo IN ({lista_grupos_valor})
) src
PIVOT (
    SUM(VL_Bem)
    FOR ST_Adimplencia IN ([Adimplente], [Inadimplente])
) AS pvt
ORDER BY Perc_Inadimplente DESC
"""

In [58]:
df_valor_anterior = pd.read_sql(query_valor_anterior, conn)
df_valor_anterior.rename(columns={
    'Tipo_Venda': 'Tipo Grupo',
    'Faixa_Credito': 'Faixa de Crédito',
    'Perc_Inadimplente': f'% Inadimplente ({data_referencia_anterior})'
}, inplace=True)
df_valor = pd.merge(df_valor, df_valor_anterior[['Grupo', 'Produto', 'Tipo Grupo', 'Faixa de Crédito', f'% Inadimplente ({data_referencia_anterior})']],
                    on=['Grupo', 'Produto', 'Tipo Grupo', 'Faixa de Crédito'], how='left')

In [59]:
df_cotas_anterior = pd.read_sql(query_cotas_anterior, conn)
df_cotas_anterior.rename(columns={
    'Tipo_Venda': 'Tipo Grupo',
    'Faixa_Credito': 'Faixa de Crédito',
    'Perc_Inadimplente': f'% Inadimplente ({data_referencia_anterior})'
}, inplace=True)

df_cotas = pd.merge(df_cotas, df_cotas_anterior[['Grupo', 'Produto', 'Tipo Grupo', 'Faixa de Crédito', f'% Inadimplente ({data_referencia_anterior})']],
                    on=['Grupo', 'Produto', 'Tipo Grupo', 'Faixa de Crédito'], how='left')

In [60]:
# Carregando tabela
script = """
select
    a.ID_Cota,
    a.Grupo,
    c.CD_InscricaoNacional as CPF_CNPJ,
    a.DT_Alocacao,
    b.DT_Contemplacao,
    a.Desclassificado
from [dbo].[FT0002_VendaAlocacoes] a
    left join [dbo].[FT0019_EntregaBem] b on a.ID_Cota = b.ID_Cota
    left join [dbo].[DM0013_Pessoas] c on a.ID_Pessoa = c.ID_Pessoa
where a.ST_Contrato = 'Ativo'
"""
df_dw = pd.read_sql(script, conn)
 
# Tratando colunas de data
df_dw['DT_Alocacao'] = pd.to_datetime(df_dw['DT_Alocacao'], format='%Y-%m-%d')
df_dw['DT_Contemplacao'] = pd.to_datetime(df_dw['DT_Contemplacao'], format='%Y-%m-%d')
df_dw['CPF_CNPJ'] = df_dw['CPF_CNPJ'].apply(trata_documento)

In [61]:
# 10.1. Base de Scores
path = fr'C:\Users\{usuario}\CAIXA Consórcio\Risco de Crédito e Antifraude - Documentos\MODELOS\MODELAGEM\SCORE_CREDITO_BOA_VISTA\BASES_HISTORICAS'
os.chdir(path)
df_score = pd.read_csv('score_credito_full.csv', delimiter=";", encoding='latin1')
df_score['Data'] = pd.to_datetime(df_score['Data'], format='%Y-%m-%d')
df_score['CPF_CNPJ'] = df_score['CPF_CNPJ'].apply(trata_documento)
df_score = df_score[['Data', 'CPF_CNPJ', 'Score_Credito']]
df_score = df_score[df_score['Score_Credito'] > 2]
 
# 10.2. Coletando Scores na Venda
df_venda = df_dw[['ID_Cota', 'Grupo', 'CPF_CNPJ', 'DT_Alocacao']]
df_venda = df_venda.merge(df_score, how='left', on=['CPF_CNPJ'])
df_venda = df_venda[df_venda['Score_Credito'].notna()]
df_venda['Variacao_Datas'] = abs(df_venda['Data'] - df_venda['DT_Alocacao']).dt.days
df_venda = df_venda.sort_values(by=['Variacao_Datas'], ascending=True)
df_venda = df_venda.drop_duplicates(subset=['ID_Cota'])
df_venda = df_venda[['Grupo', 'Score_Credito']].groupby(by=['Grupo']).mean().reset_index(level=['Grupo'])
df_venda = df_venda.rename(columns={'Score_Credito': 'Score de Crédito (Venda)'})
df_venda['Score de Crédito (Venda)'] = round(df_venda['Score de Crédito (Venda)'],0)
df_venda['Score de Crédito (Venda)'] = df_venda['Score de Crédito (Venda)'].apply(trata_documento)
 
# 10.3. Coletando Scores na Venda
df_contemplacao = df_dw[df_dw['Desclassificado'] == 'Não']
df_contemplacao = df_contemplacao[['ID_Cota', 'Grupo', 'CPF_CNPJ', 'DT_Contemplacao']]
df_contemplacao = df_contemplacao.merge(df_score, how='left', on=['CPF_CNPJ'])
df_contemplacao = df_contemplacao[df_contemplacao['Score_Credito'].notna()]
df_contemplacao['Variacao_Datas'] = abs(df_contemplacao['Data'] - df_contemplacao['DT_Contemplacao']).dt.days
df_contemplacao = df_contemplacao.sort_values(by=['Variacao_Datas'], ascending=True)
df_contemplacao = df_contemplacao.drop_duplicates(subset=['ID_Cota'])
df_contemplacao = df_contemplacao[['Grupo', 'Score_Credito']].groupby(by=['Grupo']).mean().reset_index(level=['Grupo'])
df_contemplacao = df_contemplacao.rename(columns={'Score_Credito': 'Score de Crédito (Contemplação)'})
df_contemplacao['Score de Crédito (Contemplação)'] = round(df_contemplacao['Score de Crédito (Contemplação)'],0)
df_contemplacao['Score de Crédito (Contemplação)'] = df_contemplacao['Score de Crédito (Contemplação)'].apply(trata_documento)
 
# 10.4. Juntando as duas tabelas
df_venda = df_venda.merge(df_contemplacao, how='left', on=['Grupo'])
 
# 11. Juntando as tabelas
df_cotas = df_cotas.merge(df_venda, how='left', on=['Grupo'])
df_valor = df_valor.merge(df_venda, how='left', on=['Grupo'])

In [62]:
# 13. Calcular e formatar variação
df_valor[f'Variação % Inadimplente ({data_referencia})'] = (
    df_valor[f'% Inadimplente ({data_referencia})'].apply(percentual_para_float) -
    df_valor[f'% Inadimplente ({data_referencia_anterior})'].apply(percentual_para_float)
).apply(lambda x: f"{x:+.1f}".replace('.', ','))


 
df_cotas[f'Variação % Inadimplente ({data_referencia})'] = (
    df_cotas[f'% Inadimplente ({data_referencia})'].apply(percentual_para_float) -
    df_cotas[f'% Inadimplente ({data_referencia_anterior})'].apply(percentual_para_float)
).apply(lambda x: f"{x:+.1f}".replace('.', ','))
 
 
# 14. Exportar Excel
caminho_pasta = fr'C:\Users\{usuario}\CAIXA Consórcio\Risco de Crédito e Antifraude - Documentos\CRÉDITO\INADIMPLÊNCIA\INDICADORES'
os.makedirs(caminho_pasta, exist_ok=True)
nome_arquivo = f'Monitoramento_Inadimplência_Mensal_{data_referencia}.xlsx'
caminho_arquivo = os.path.join(caminho_pasta, nome_arquivo)
 
with pd.ExcelWriter(caminho_arquivo, engine='openpyxl') as writer:
    df_valor.to_excel(writer, sheet_name='Por Valor', index=False)
    df_cotas.to_excel(writer, sheet_name='Por Cotas', index=False)
 
# 15. HTML com linhas em branco
def df_to_html(df):  # <- aqui o argumento df precisa estar presente
    header_color = "rgb(0,32,96)"
    row_color = "rgb(255,255,255)"
    title_font_size = "16px"
    row_font_size = "17px"
    min_column_width = "120px"

    # Pega apenas os 20 primeiros registros
    df = df.head(20)

    html = f"""
    <table border="1" cellspacing="0" cellpadding="5" style="border-collapse:collapse; font-family:Arial; font-size:{row_font_size}; text-align:center; width:100%;">
        <thead>
            <tr style="background-color:{header_color}; color:white; text-align:center; font-size:{title_font_size};">
    """
    for col in df.columns:
        html += f"<th style='padding: 8px; min-width:{min_column_width};'>{col}</th>"
    html += "</tr></thead><tbody>"

    for _, row in df.iterrows():
        html += f'<tr style="background-color:{row_color};">'
        for i, cell in enumerate(row):
            if i == len(row) - 1:  # Última coluna: variação
                valor = str(cell).replace(',', '.').replace('%', '')
                try:
                    numero = float(valor)
                    if numero < 0:
                        simbolo = "▼"
                        cor = "green"
                    else:
                        simbolo = "▲"
                        cor = "red"
                    cell_formatado = f'<span style="color:{cor}; font-weight:bold; font-size:18px;">{simbolo} {cell}</span>'
                except:
                    cell_formatado = cell
                html += f'<td style="padding:8px;">{cell_formatado}</td>'
            else:
                html += f'<td style="padding:8px;">{cell}</td>'
        html += "</tr>"
    html += "</tbody></table>"
    return html

 
# 15.1 Carrega a assinatura HTML externa
with open(fr'C:\Users\{usuario}\CAIXA Consórcio\Risco de Crédito e Antifraude - Documentos\ADMINISTRATIVO\APRESENTAÇÕES\assinatura_principal_Barueri.html', 'r', encoding='utf-8') as file:
    html_assinatura = file.read()

In [63]:
df_valor.rename(columns={
    'Score de Crédito (Venda)': 'Score Venda',
    'Score de Crédito (Contemplação)': 'Score Contemplação',
    'R$ MM Total': 'R$ MM Total'
}, inplace=True)
 
df_valor['R$ MM Total'] = (
    df_valor['R$ MM Adimplente'].str.replace('.', '').str.replace(',', '.').astype(float) +
    df_valor['R$ MM Inadimplente'].str.replace('.', '').str.replace(',', '.').astype(float)
).apply(lambda x: f"{x:,.1f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
 
# Ordenar colunas
cols_valor = [
    'Grupo', 'Produto', 'Tipo Grupo', 'Faixa de Crédito',
    'Score Venda', 'Score Contemplação',
    'R$ MM Total', 'R$ MM Adimplente', 'R$ MM Inadimplente',
    f'% Inadimplente ({data_referencia_anterior})',
    f'% Inadimplente ({data_referencia})',
    f'Variação % Inadimplente ({data_referencia})'
]
df_valor = df_valor[cols_valor]
 
#--------------------------------
#--------------------------------
 
df_cotas.rename(columns={
    'Score de Crédito (Venda)': 'Score Venda',
    'Score de Crédito (Contemplação)': 'Score Contemplação',
    'Qtd. Cotas Adimplentes': 'Qtd. Cotas Adim.',
    'Qtd. Cotas Inadimplentes': 'Qtd. Cotas Inad.'
}, inplace=True)
 
cols_cotas = [
    'Grupo', 'Produto', 'Tipo Grupo', 'Faixa de Crédito',
    'Score Venda', 'Score Contemplação',
    'Qtd. Clientes', 'Qtd. Cotas Adim.', 'Qtd. Cotas Inad.',
    f'% Inadimplente ({data_referencia_anterior})',
    f'% Inadimplente ({data_referencia})',
    f'Variação % Inadimplente ({data_referencia})'
]
df_cotas = df_cotas[cols_cotas]

In [64]:
html_tabela_valor = df_to_html(df_valor)
html_tabela_cotas = df_to_html(df_cotas)

outlook = win32.Dispatch('Outlook.Application')
email = outlook.CreateItem(0)

# Configurações principais
email.To = "gabriel.silva@caixaconsorcio.com.br"
email.Subject = "Monitoramento Mensal da Inadimplente"

# Caso você tenha permissão de "Send on behalf"
email.SentOnBehalfOfName = "modelagemriscoecredito@caixaconsorcio.com.br"

# Corpo do e-mail em HTML
email.HTMLBody = f"""
    <p>Prezados,</p>
 
<p>Segue abaixo o monitoramento da inadimplência referente ao período de <b>{data_referencia}</b>, considerando duas perspectivas complementares: valores de carta de crédito e quantidade de cotas distintas.</p> <p>Os campos <b>Grupo</b>, <b>Produto</b>, <b>Tipo Grupo</b> e <b>Faixa de Crédito</b> identificam e caracterizam os grupos analisados. Na visão por carta de crédito, são apresentados os valores adimplentes e inadimplentes em milhões de reais, permitindo o cálculo do <b>% Inadimplente</b>, que representa a proporção inadimplente sobre o total contratado. Na visão por cotas, são consideradas as quantidades de cotas distintas adimplentes e inadimplentes, com o mesmo indicador percentual aplicado à base de cotas.</p> <p>Ambas as visões trazem ainda a <b>Variação % Inadimplente</b>, comparando o percentual atual com o do mês anterior, de modo a evidenciar a evolução do comportamento de pagamento: reduções sinalizam melhora, enquanto aumentos indicam deterioração.</p>
</ul>
<br>
<ul>
  <li><strong>📊 Visão por Carta de Crédito:</strong></li>
</ul>

{html_tabela_valor}

<br>
<br>
<ul>
  <li><strong>📉 Visão por Cotas Distintas:</strong></li>
</ul>

{html_tabela_cotas}
 
<p>O arquivo em Excel também está em anexo com todos os dados.</p>
 
<p>A equipe de Modelagem e Risco de Crédito fica à disposição para qualquer dúvida ou esclarecimento adicional.</p>
{html_assinatura}
"""

# Anexa o arquivo
email.Attachments.Add(Source=caminho_arquivo)

# Envia o e-mail
email.Send()
